In [81]:
import gc
import os
import time
import warnings
from tqdm import tqdm
from typing import List, Optional

import shap
import numpy as np
import pandas as pd
import catboost as cb
import xgboost as xgb
import lightgbm as lgbm

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

warnings.simplefilter("ignore")
pd.set_option("display.max_columns", 100)
shap.initjs()

## Useful Functions

In [82]:
def get_input(data_path: str) -> pd.DataFrame:
    """
    Считывание данных и вывод основной информации о наборе данных.

    Parameters
    ----------
    data_path: str
        Название файла.

    Returns
    -------
    data: pandas.core.frame.DataFrame
        Загруженный набор данных в pandas.DataFrame

    """
    base_path = "geekbrains-competitive-data-analysis"
    data = pd.read_csv(f"{base_path}/{data_path}")
    data.columns = [col.lower() for col in data.columns]
    print(f"{data_path}: shape = {data.shape[0]} rows, {data.shape[1]} cols")

    return data


def lightgbm_cross_validation(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели catbooost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, folds_scores = [], []
    oof_preds = np.zeros(X.shape[0])
    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = lgbm.LGBMClassifier(**params)
        model.fit(
            x_train, y_train,
            eval_set=[(x_valid, y_valid)],
            eval_metric="auc", verbose=50, early_stopping_rounds=50,
            categorical_feature=categorical
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)[:, 1]
        score = roc_auc_score(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds


def catboost_cross_validation(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели catbooost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, folds_scores = [], []
    oof_preds = np.zeros(X.shape[0])

    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")
    X[categorical] = X[categorical].astype(str)

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = cb.CatBoostClassifier(**params)
        model.fit(
            x_train, y_train, categorical,
            eval_set=[(x_train, y_train), (x_valid, y_valid)]
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)[:, 1]
        score = roc_auc_score(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds


def xgboost_cross_validation(params, X, y, cv, categorical = None):
    """
    Кросс-валидация для модели xgboost.

    Parameters
    ----------
    params: dict
        Словарь гиперпараметров модели.

    X: pandas.core.frame.DataFrame
        Матрица признако для обучения модели.

    y: pandas.core.frame.Series
        Вектор целевой переменной для обучения модели.

    cv: KFold or StratifiedKFold generator.
        Объект KFold / StratifiedKFold для определения
        стратегии кросс-валидации модели.

    categorical: str, optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    estimators: list
        Список с объектами обученной модели.

    oof_preds: np.array
        Вектор OOF-прогнозов.

    """
    estimators, folds_scores = [], []
    oof_preds = np.zeros(X.shape[0])

    print(f"{time.ctime()}, Cross-Validation, {X.shape[0]} rows, {X.shape[1]} cols")
    X[categorical] = X[categorical].astype(str)

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X, y)):

        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = xgb.XGBClassifier(**params)
        model.fit(
            x_train, y_train,
            eval_metric="auc",
            eval_set=[(x_train, y_train), (x_valid, y_valid)],
            early_stopping_rounds=50,
            verbose=25
        )
        oof_preds[valid_idx] = model.predict_proba(x_valid)[:, 1]
        score = roc_auc_score(y_valid, oof_preds[valid_idx])
        print(f"Fold {fold+1}, Valid score = {round(score, 5)}")
        folds_scores.append(round(score, 5))
        estimators.append(model)

    print(f"Score by each fold: {folds_scores}")
    print("="*65)
    return estimators, oof_preds

## Feature Engineering

In [83]:
def create_numerical_aggs(data: pd.DataFrame,
                          groupby_id: str,
                          aggs: dict,
                          prefix: Optional[str] = None,
                          suffix: Optional[str] = None,
                          ) -> pd.DataFrame:
    """
    Построение агрегаций для числовых признаков.

    Parameters
    ----------
    data: pandas.core.frame.DataFrame
        Выборка для построения агрегаций.

    groupby_id: str
        Название ключа, по которому нужно произвести группировку.

    aggs: dict
        Словарь с названием признака и списка функций.
        Ключ словаря - название признака, который используется для
        вычисления агрегаций, значение словаря - список с названием
        функций для вычисления агрегаций.

    prefix: str, optional, default = None
        Префикс для названия признаков.
        Опциональный параметр, по умолчанию, не используется.

    suffix: str, optional, default = None
        Суффикс для названия признаков.
        Опциональный параметр, по умолчанию, не используется.

    Returns
    -------
    stats: pandas.core.frame.DataFrame
        Выборка с рассчитанными агрегациями.

    """
    if not prefix:
        prefix = ""
    if not suffix:
        suffix = ""

    data_grouped = data.groupby(groupby_id)
    stats = data_grouped.agg(aggs)
    stats.columns = [f"{prefix}{feature}_{stat}{suffix}".lower() 
                     for feature, stat in stats]
    stats = stats.reset_index()

    return stats

def create_client_profile_features(X: pd.DataFrame, copy: bool = True) -> pd.DataFrame:
    """
    Создание признаков на основе профиля клиентов.

    Parameters
    ----------
    X: pandas.core.frame.DataFrame
        Матрица признаков с исходным профилем клиента.

    copy: bool, optional, default = True
        Флаг использования копии датафрейма X.
        Опциональный параметр, по умолчанию, равен True.

    Returns
    -------
    X_transformed: pandas.core.frame.DataFrame
        Расширенная матрица признаков с профилем клиентов.

    """
    if copy:
        X = X.copy()

    X["days_on_last_job"] = X["days_on_last_job"].replace(365243, np.nan)
    bki_flags = [flag for flag in X.columns if "amt_req_credit_bureau" in flag]
    X["bki_requests_count"] = X[bki_flags].sum(axis=1)
    X["bki_kurtosis"] = X[bki_flags].kurtosis(axis=1)

    X["external_scoring_prod"] = X["external_scoring_rating_1"] * X["external_scoring_rating_2"] * X["external_scoring_rating_3"]
    X["external_scoring_weighted"] = X.external_scoring_rating_1 * 2 + X.external_scoring_rating_2 * 1 + X.external_scoring_rating_3 * 3

    for function_name in ["min", "max", "mean", "sum", "nanmedian"]:
        feature_name = "external_scoring_rating_{}".format(function_name)
        X[feature_name] = eval("np.{}".format(function_name))(
            X[["external_scoring_rating_1", "external_scoring_rating_2", "external_scoring_rating_3"]], axis=1
        )

    # Отношение между основными фин. показателями
    X['ratio_credit_to_annuity'] = X['amount_credit'] / X['amount_annuity']
    X["ratio_annuity_to_salary"] = X['amount_annuity'] / X['total_salary']
    X["payment_rate"] = X['amount_annuity'] / X['amount_credit']
    #X["total_salary_net"] = X["total_salary"] - X["amount_annuity"]

    # Отношение фин. показателей к возрасту и временным фичам
    X['ratio_credit_to_salary'] = X['amount_credit'] / X['total_salary']
    X["ratio_annuity_to_age"] = X["amount_annuity"] / X["age"]
    X["ratio_credit_to_age"] = X["amount_credit"] / X["age"]
    X["ratio_salary_to_age"] = X["total_salary"] / X["age"]
    X["ratio_salary_to_experience"] = X["total_salary"] / X["days_on_last_job"]
    X["ratio_credit_to_experience"] = X["amount_credit"] / X["days_on_last_job"]
    X["ratio_annuity_to_experience"] = X["amount_annuity"] / X["days_on_last_job"]

    # Отношение врменных признаков
    X["ratio_age_to_experience"] = X["age"] / X["days_on_last_job"]
    X["ratio_salary_to_region_population"] = X["total_salary"] * X["region_population"]
    X["ratio_car_to_experience"] = X["own_car_age"] / X["days_on_last_job"]
    X["ratio_car_to_age"] = X["own_car_age"] / X["age"]

    # Произведение фин. показателей кредита на вероятность дефолта
    # Такая штука называется математическим ожиданием дефолта или ожидаемыми потерями
    X["expected_total_loss_1"] = X["external_scoring_rating_1"] * X["amount_credit"]
    X["expected_total_loss_2"] = X["external_scoring_rating_2"] * X["amount_credit"]
    X["expected_total_loss_3"] = X["external_scoring_rating_3"] * X["amount_credit"]
    X["expected_monthly_loss_1"] = X["external_scoring_rating_1"] * X["amount_annuity"]
    X["expected_monthly_loss_2"] = X["external_scoring_rating_2"] * X["amount_annuity"]
    X["expected_monthly_loss_3"] = X["external_scoring_rating_3"] * X["amount_annuity"]

    X["external_scoring_rating_1_plus_2"] = np.nansum(
        X[["external_scoring_rating_1", "external_scoring_rating_2"]], axis=1
    )
    X["external_scoring_rating_1_plus_3"] = np.nansum(
        X[["external_scoring_rating_1", "external_scoring_rating_3"]], axis=1
    )
    X["external_scoring_rating_2_plus_3"] = np.nansum(
        X[["external_scoring_rating_2", "external_scoring_rating_3"]], axis=1
    )
    X["external_scoring_rating_1_is_nan"] = np.isnan(
        X["external_scoring_rating_1"]
    )
    X["external_scoring_rating_2_is_nan"] = np.isnan(
        X["external_scoring_rating_2"]
    )
    X["external_scoring_rating_3_is_nan"] = np.isnan(
        X["external_scoring_rating_3"]
    )

    # дополнительные признаки
    X["ratio_credit_per_family_size"] = X["amount_credit"] / X["family_size"]
    X["ratio_credit_per_childrens"] = X["amount_credit"] / (1 + X["childrens"])
    X["ratio_children_to_family_size"] = X["childrens"] / X["family_size"]
    X["ratio_salary_per_family_size"] = X["total_salary"] / X["family_size"]
    X["ratio_salary_per_child"] = X["total_salary"] / X["childrens"]
    X["non_child"] = X["family_size"] - X["childrens"]
    X["ratio_child_to_non_child"] = X["childrens"] / X["non_child"]
    X["ratio_salary_per_non_child"] = X["total_salary"] / X["non_child"]
    X["ratio_credit_per_non_child"] = X["amount_credit"] / X["non_child"]

    return X


def encode_categorical(X: pd.DataFrame, categorical: List[str] = None, copy: bool = True) -> pd.DataFrame:
    """
    Преобразование категориальных признаков с
    помощью LabelEncoder.

    Parameters
    ----------
    X: pandas.core.frame.DataFrame
        Матрица признаков.

    categorical: List[str], optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, равен None.

    copy: bool, optional, default = True
        Флаг использования копии датафрейма X.
        Опциональный параметр, по умолчанию, равен True.

    Returns
    -------
    X_transformed: pandas.core.frame.DataFrame
        Преобразованная матрица признаков.

    """
    if copy:
        X = X.copy()

    if categorical:
        for feature in categorical:
            encoder = LabelEncoder()
            X[f"{feature}_categorical"] = encoder.fit_transform(
                X[feature].fillna("NA").astype("str")
            )
        return X

    return X


def create_hot_one_encoded_features(X: pd.DataFrame, categorical: List[str] = None):
    """
    Кодирование текстовых переменных с помощью One-Hot-Encoding.

    Parameters
    ----------
    X: pandas.DataFrame
        Сырые данные;

    categorical: List[str], optional, default = None
        Список категориальных признаков.
        Опциональный параметр, по умолчанию, равен None.

    Returns
    -------
    encoded_features: pandas.DataFrame
        Матрица OHE - признаков.

    """
    encoded_features = pd.get_dummies(X[categorical])
    categorical_column_names = [col.lower().replace(" ", "_")
                                for col in encoded_features.columns]
    encoded_features.columns = categorical_column_names

    return encoded_features, categorical_column_names


def prepare_data_for_fit(X: pd.DataFrame):
    """
    Подготовка данных для обучения: разбиение данных на обучающую
    часть (где есть значения целевой переменной) и тестовую часть (где
    таких значений нет), удаление ненужных признаков для обучения и
    выявление категориальных переменных.

    Parameters
    ----------
    X: pandas.core.frame.DataFrame
        Матрица признаков.

    Returns
    -------
    x_train: pandas.core.frame.DataFrame
        Матрица признаков для обучения алгоритма.

    x_test: pandas.core.frame.DataFrame
        Матрица признаков для предсказания алгоритма.

    target: pandas.core.frame.Series
        Вектор целевой переменной.

    categorical: List[str]
        Название категориальных признаков после обработки.

    """
    mask = X["target"].notnull()
    categorical = X.select_dtypes(include=["object"]).columns.tolist()

    X = encode_categorical(X, categorical)
    categorical_new = [col for col in X.columns if "categorical" in col]
    to_drop = categorical + ["application_number", "target"]

    x_train = X.loc[mask]
    x_test = X.loc[~mask]
    target = x_train["target"]

    x_train = x_train.drop(
        to_drop, axis=1
    )
    x_test = x_test.drop(
        to_drop, axis=1
    )
    return x_train, x_test, target, categorical_new

In [84]:
class BaseFeatureTransformer(BaseEstimator, TransformerMixin):
    """
    Базовый метод для работы с сырыми данными.
    Реализует методы построения агрегаций для числовых признаков.

    """
    
    @staticmethod
    def create_numerical_aggs(data: pd.DataFrame,
                              groupby_id: str,
                              aggs: dict,
                              prefix: Optional[str] = None,
                              suffix: Optional[str] = None,
                              ) -> pd.DataFrame:
        """
        Построение агрегаций для числовых признаков.

        Parameters
        ----------
        data: pandas.core.frame.DataFrame
            Выборка для построения агрегаций.

        groupby_id: str
            Название ключа, по которому нужно произвести группировку.

        aggs: dict
            Словарь с названием признака и списка функций.
            Ключ словаря - название признака, который используется для
            вычисления агрегаций, значение словаря - список с названием
            функций для вычисления агрегаций.

        prefix: str, optional, default = None
            Префикс для названия признаков.
            Опциональный параметр, по умолчанию, не используется.

        suffix: str, optional, default = None
            Суффикс для названия признаков.
            Опциональный параметр, по умолчанию, не используется.

        Returns
        -------
        stats: pandas.core.frame.DataFrame
            Выборка с рассчитанными агрегациями.

        """
        if not prefix:
            prefix = ""
        if not suffix:
            suffix = ""

        data_grouped = data.groupby(groupby_id)
        stats = data_grouped.agg(aggs)
        stats.columns = [f"{prefix}{feature}_{stat}{suffix}".lower() 
                         for feature, stat in stats]
        stats = stats.reset_index()

        return stats


class BureauFeaturesTransformer(BaseFeatureTransformer):

    def __init__(self, copy: bool = True):
        self.copy = copy

    def _preprocessing_data(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        TBD
        """
        X = X.sort_values(by=["application_number", "days_credit"], ascending=False)
        X["bureau_credit_active"] = (X["credit_active"] == "Closed").astype(int)
        X["bureau_credit_enddate_binary"] = (X["days_credit_enddate"] > 0).astype(int)
        X["bureau_credit_type_consumer"] = (X["credit_type"] == "Consumer credit").astype(int)
        X["bureau_credit_type_car"] = (X["credit_type"] == "Car loan").astype(int)
        X["bureau_credit_type_mortgage"] = (X["credit_type"] == "Mortgage").astype(int)
        X["bureau_credit_type_credit_card"] = (X["credit_type"] == "Credit card").astype(int)
        X["bureau_credit_type_other"] = (~(
            X["credit_type"].isin(["Consumer credit", "Car loan", "Morgage", "Credit card"])
        )).astype(int)
        X["bureau_unusual_currency"] = (~(X["credit_currency"] == "currency 1")).astype(int)
        X["days_credit_diff"] = X["days_credit"].diff().replace(np.nan, 0)
        
        return X

    def _create_groupby_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        TBD
        """
        aggs = {
            "days_credit": ["count"],
            "credit_type": ["nunique"],
            "amt_credit_sum_debt": ["sum"],
            "amt_credit_sum": ["sum"],
            "amt_credit_sum_overdue": ["sum"],
            "cnt_credit_prolong": ["sum"],
            "bureau_credit_enddate_binary": ["mean"],
            #"bureau_credit_active": ["sum", "mean"],
        }
        features = pd.DataFrame({
            "application_number": X["application_number"].unique()
        })
        groupby_features = self.create_numerical_aggs(
            X, "application_number", aggs=aggs, prefix="bureau_"
        )
        features = features.merge(
            groupby_features, how="left", on="application_number"
        )
        return features

    def _create_ratio_features(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        TBD
        """
        X["bureau_average_of_past_loans_per_type"] = \
            X["bureau_days_credit_count"] / X["bureau_credit_type_nunique"]
        X["bureau_debr_credit_ratio"] = \
            X["bureau_amt_credit_sum_debt_sum"] / X["bureau_amt_credit_sum_sum"]
        X["bureau_overdue_debt_ratio"] = \
            X["bureau_amt_credit_sum_overdue_sum"] / X["bureau_amt_credit_sum_debt_sum"]

        return X

    def transform(self, X, y=None) -> pd.DataFrame:
        if self.copy:
            X = X.copy()

        X = self._preprocessing_data(X)
        X_grouped = X.groupby(["application_number"])

        features = self._create_groupby_features(X)
        features = self._create_ratio_features(features)

        return features

In [ ]:
# тренды (сырые данные)
# История заявок (prev_applications.csv)
# Для каждого клиента:
#     - взять его предыдущие заявки
#     - отсортировать по времени (от старых к новым)
#     - выбрать конкретный признак (например: сумма кредита в заявке)
#     - обучить линейную регрессию на одном признаке: x - номер заявки

In [153]:
apps = get_input("applications_history.csv")
apps.head(n=2)

applications_history.csv: shape = 1670214 rows, 26 cols


prev_application_number  application_number name_contract_type  \
0                 49298709           123595216                NaN   
1                 50070639           123431468               Cash   

   amount_annuity  amt_application  amount_credit  amount_payment  \
0        1730.430          17145.0        17145.0             0.0   
1       25188.615         607500.0       679671.0             NaN   

   amount_goods_payment name_contract_status  days_decision  \
0               17145.0             Approved             73   
1              607500.0             Approved            164   

       name_payment_type code_reject_reason name_type_suite name_client_type  \
0  Cash through the bank                XAP             NaN         Repeater   
1                    XNA                XAP   Unaccompanied         Repeater   

  name_goods_category name_portfolio name_product_type  sellerplace_area  \
0              Mobile            POS               XNA                35   
1                 XNA           Cash            x-sell                -1   

   cnt_payment name_yield_group  days_first_drawing  days_first_due  \
0         12.0           middle            365243.0            42.0   
1         36.0       low_action            365243.0           134.0   

   days_last_due_1st_version  days_last_due  days_termination  \
0                      300.0           42.0              37.0   
1                      916.0       365243.0          365243.0   

   nflag_insured_on_approval  
0                        0.0  
1                        1.0

In [157]:
from sklearn.linear_model import LinearRegression

In [164]:
trend_data = apps[apps["application_number"]==123595216]
trend_data = trend_data.sort_values(by="days_decision")["amt_application"]

model = LinearRegression()
model.fit(np.arange(trend_data.shape[0]).reshape(-1, 1), trend_data.values)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [167]:
model.intercept_

596220.7500000001

## Create Training Dataset

In [85]:
train = get_input("train.csv")
test = get_input("test.csv")

data = pd.concat([train, test], axis=0)
data = data.reset_index(drop=True)

client_profile = get_input("client_profile.csv")
client_profile = create_client_profile_features(client_profile)

data = data.merge(
    client_profile, how="left", on="application_number"
)

bki = get_input("bki.csv")
bki_features = BureauFeaturesTransformer().transform(X=bki)
data = data.merge(
    bki_features, how="left", on="application_number"
)

x_train, x_test, target, categorical_new = prepare_data_for_fit(data)

train.csv: shape = 110093 rows, 3 cols
test.csv: shape = 165141 rows, 2 cols
client_profile.csv: shape = 250000 rows, 24 cols
bki.csv: shape = 945234 rows, 17 cols


## LightGBM

In [86]:
lgbm_params = {
    'boosting_type': 'gbdt',
    'n_estimators': 10000,
    'learning_rate': 0.005134,
    'num_leaves': 54,
    'max_depth': 10,
    'subsample_for_bin': 240000,
    'reg_alpha': 0.436193,
    'reg_lambda': 0.479169,
    'colsample_bytree': 0.508716,
    'min_split_gain': 0.024766,
    'subsample': 0.7,
    'is_unbalance': False,
    'random_state': 42,
    'silent': -1,
    'verbose': -1
}

cv = KFold(n_splits=7, random_state=435, shuffle=True)

In [7]:
estimators, oof_preds = lightgbm_cross_validation(
    lgbm_params, x_train, target, cv=cv, categorical=categorical_new
)

Tue Oct  6 16:31:23 2020, Cross-Validation, 110093 rows, 78 cols
Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.720812	valid_0's binary_logloss: 0.270663
[100]	valid_0's auc: 0.724714	valid_0's binary_logloss: 0.264669
[150]	valid_0's auc: 0.727282	valid_0's binary_logloss: 0.260831
[200]	valid_0's auc: 0.728968	valid_0's binary_logloss: 0.258208
[250]	valid_0's auc: 0.730196	valid_0's binary_logloss: 0.256323
[300]	valid_0's auc: 0.731741	valid_0's binary_logloss: 0.254947
[350]	valid_0's auc: 0.732872	valid_0's binary_logloss: 0.253851
[400]	valid_0's auc: 0.73369	valid_0's binary_logloss: 0.253051
[450]	valid_0's auc: 0.735812	valid_0's binary_logloss: 0.25232
[500]	valid_0's auc: 0.737034	valid_0's binary_logloss: 0.251811
[550]	valid_0's auc: 0.738997	valid_0's binary_logloss: 0.25132
[600]	valid_0's auc: 0.740023	valid_0's binary_logloss: 0.250901
[650]	valid_0's auc: 0.740806	valid_0's binary_logloss: 0.250539
[700]	valid_0's auc: 0.74152	vali

[800]	valid_0's auc: 0.714264	valid_0's binary_logloss: 0.256981
[850]	valid_0's auc: 0.714591	valid_0's binary_logloss: 0.25691
[900]	valid_0's auc: 0.715014	valid_0's binary_logloss: 0.256874
[950]	valid_0's auc: 0.715136	valid_0's binary_logloss: 0.256845
[1000]	valid_0's auc: 0.715206	valid_0's binary_logloss: 0.256798
[1050]	valid_0's auc: 0.715495	valid_0's binary_logloss: 0.256756
[1100]	valid_0's auc: 0.715493	valid_0's binary_logloss: 0.256754
Early stopping, best iteration is:
[1059]	valid_0's auc: 0.715607	valid_0's binary_logloss: 0.256741
Fold 6, Valid score = 0.71561
Training until validation scores don't improve for 50 rounds
[50]	valid_0's auc: 0.70514	valid_0's binary_logloss: 0.264702
[100]	valid_0's auc: 0.707295	valid_0's binary_logloss: 0.259872
[150]	valid_0's auc: 0.708624	valid_0's binary_logloss: 0.256758
[200]	valid_0's auc: 0.70984	valid_0's binary_logloss: 0.254661
[250]	valid_0's auc: 0.711931	valid_0's binary_logloss: 0.253186
[300]	valid_0's auc: 0.713697

In [8]:
oof_score = roc_auc_score(
    target, oof_preds
)
print(f"OOF-score = {round(oof_score, 5)}")

# OOF-score = 0.7269
# Score by each fold: [0.74345, 0.74354, 0.73298, 0.71882, 0.71979, 0.71561, 0.72341]

OOF-score = 0.7269


## Final Model

In [140]:
features_to_drop = [
    'amt_req_credit_bureau_day',
    'flag_email',
    'ratio_children_to_family_size',
    'external_scoring_rating_3_is_nan_categorical',
    'childrens',
    'ratio_child_to_non_child',
    'bureau_overdue_debt_ratio',
    'bki_requests_count',
    'bureau_amt_credit_sum_debt_sum',
    'bureau_amt_credit_sum_overdue_sum',
    'external_scoring_rating_sum',
    'bureau_credit_type_nunique',
    'expected_monthly_loss_1',
    'ratio_credit_per_family_size',
    'ratio_salary_per_family_size',
    'ratio_car_to_experience',
    'bureau_debr_credit_ratio',
    'total_salary'
]

In [141]:
lgb_model = lgbm.LGBMClassifier(**lgbm_params)
lgb_model.n_estimators = 1000

lgb_model.fit(x_train.drop(features_to_drop, axis=1), target)

LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.508716, importance_type='split',
               is_unbalance=False, learning_rate=0.005134, max_depth=10,
               min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.024766, n_estimators=1000, n_jobs=-1,
               num_leaves=54, objective=None, random_state=42,
               reg_alpha=0.436193, reg_lambda=0.479169, silent=-1,
               subsample=0.7, subsample_for_bin=240000, subsample_freq=0,
               verbose=-1)

## Pseudo Labeling

In [142]:
y_test_ = (y_pred > 0.5).astype(int)

x_test_ = x_test.copy()
x_test_["TARGET"] = y_test_
x_test_ = x_test_[x_test_["TARGET"] == 1]

x_train_ = pd.concat([x_train, x_test_.drop("TARGET", axis=1)])
target_ = pd.concat([target, x_test_["TARGET"]])

In [147]:
lgb_model = lgbm.LGBMClassifier(**lgbm_params)
lgb_model.n_estimators = 1500

lgb_model.fit(x_train_, target_)
y_pred = lgb_model.predict_proba(x_test)[:, 1]

LGBMClassifier(boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.508716, importance_type='split',
               is_unbalance=False, learning_rate=0.005134, max_depth=10,
               min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.024766, n_estimators=1500, n_jobs=-1,
               num_leaves=54, objective=None, random_state=42,
               reg_alpha=0.436193, reg_lambda=0.479169, silent=-1,
               subsample=0.7, subsample_for_bin=240000, subsample_freq=0,
               verbose=-1)

## Final Submit

In [150]:
submit = y_test.copy()
submit["TARGET"] = y_pred
submit.to_csv("./geekbrains-competitive-data-analysis/score_6october_pseudo_final.csv", index=False)